# Ejercicio 1

En este primer ejercicio procedemos a indexar la colección de documentos `train.docs`, dicha colección será tokenizada y se creará un índice BM25.

Primero empezamos importando las dependencias necesarias:


In [ ]:
import bm25s
import Stemmer

Estas librerías nos permiten:
   - bm25s: Crear índices y buscar documentos usando BM25
   - Stemmer: Realizar stemming de los términos

Una vez importadas dichas librerías necesitamos carga el corpus para ello se define la siguiente función:

In [ ]:
def prepareCorpus():

    # Cargamos el archivo train.docs, y lo separamos por el salto de línea
    with open("../archivos/train.docs","r",encoding="utf-8") as f:
        corpus_content = f.read()
        corpus_content = corpus_content.split("\n")

    #Inicializamos unas listas vacías
    corpus_plainText = list()
    corpus_verbatim = list()

    #Recorremos el corpus_content y lo dividimos en tres partes, el id, el título y el texto
    for valor in corpus_content:
        partes = valor.split("\t")
        valor = partes[1].lower()
        document = {"id": partes[0], "title":partes[0].lower(),"text": valor}
        corpus_verbatim.append(document)
        corpus_plainText.append(valor)

    #Inicializamos el stemmer
    stemmer = Stemmer.Stemmer("english")

    #Tokenizamos el corpus
    corpus_tokenized = bm25s.tokenize(corpus_plainText, stopwords="en", stemmer=stemmer, show_progress=True)

    #Creamos el índice BM25 y lo guardamos en un archivo
    retriever = createAndSaveRetriever(corpus_verbatim, corpus_tokenized)


La función anterior lee el archivo `train.docs` y lo divide en tres partes, la primera parte es el id<sup>1</sup> del documento, la segunda es el título del documento y la tercera parte es el texto del documento. Luego se tokeniza el texto del documento y se guarda en un índice BM25.

Este método llama a la función `createAndSaveRetriever()` que se encarga de crear el índice BM25 y guardarlo en un archivo.

<sup>1</sup> El id es el mismo valor que el título presente en `train.docs`

---
La función `createAndSaveRetriever()` realiza lo siguiente:

1. Crea un retriever BM25 con el corpus verbatim y los métodos de cálculo de BM25 y de idf.
2. Indexa el corpus tokenizado.
3. Guarda el retriever en un archivo, para posteriormente ser recuperado

In [ ]:
def createAndSaveRetriever(corpus_verbatim, corpus_tokenized,method="lucene",idf_method="lucene"):

    #Creacion del retriever
    retriever = bm25s.BM25(corpus=corpus_verbatim, method=method, idf_method=idf_method)
    #Indexado de la colección de documentos tokenizados
    retriever.index(corpus_tokenized, show_progress=True)

    #Guardado del retriever en un archivo
    retriever.save("../archivos/NFcorpus",corpus=corpus_verbatim)

    return retriever

---
En este documento también hay presente una función para poder cargar el retriever de un archivo, dicha función es `openRetriever()`:

In [ ]:
def openRetriever():
    retriever = bm25s.BM25.load("../archivos/NFcorpus",load_corpus=True)
    return retriever

# Ejercicio 2

En este segundo ejercicio se procede a cargar las queries y procesarlas con diferentes modelos de búsqueda para el retriever, y realizando dichas consultas con la combinación del steamer y stopwords<sup>1</sup>,para posteriormente poder calcular la precisión de las consultas y guardarlas en un archivo.

<sup>1</sup>Se harán combinaciones de steamer y stopwords para poder comparar los resultados, dichas combinaciones son y para cada modelo de retriever: stopword-steaming, stopword-none-steaming, none-stopword-steaming, none-stopword-none-steaming.

Primero empezaremos cargando y preparando el corpus, para ello se utilizará la función `prepareCorpus()` y `createRetriever()` usadas en el ejercicio anterior y presentes en el archivo python del ejercicio 2. También se usará la función de `tokenizado()`, para poder tokenizar el corpus y las consultas según diferentes configuraciones

In [ ]:
def tokenizado(texto, stemmer, stopwords):
    # Tokenizamos el corpus
    corpus_tokenized = bm25s.tokenize(texto, stopwords=stopwords, stemmer=stemmer, show_progress=True)

    return corpus_tokenized

### Tabla de resultados
#### Macro promedios

|  Modelo   | Steamer | Stopwords | Precision | Recall |    F1     |
|:---------:|:-------:|:---------:|:---------:|:------:|:---------:|
| Robertson |   No    |    No     |   0.064   | 0.261  | __0.101__ |
| Robertson |   No    |    Si     |   0.066   | 0.265  | __0.103__ |
| Robertson |   Si    |    No     |   0.067   | 0.273  | __0.105__ |
| Robertson |   Si    |    Si     |   0.069   | 0.279  | __0.106__ |
|   Atire   |   No    |    No     |   0.064   | 0.261  |  __0.1__  |
|   Atire   |   No    |    Si     |   0.066   | 0.265  | __0.102__ |
|   Atire   |   Si    |    No     |   0.067   | 0.273  | __0.104__ |
|   Atire   |   Si    |    Si     |   0.069   | 0.278  | __0.107__ |
|   Bm25l   |   No    |    No     |   0.064   | 0.263  | __0.101__ |
|   Bm25l   |   No    |    Si     |   0.066   | 0.266  | __0.102__ |
|   Bm25l   |   Si    |    No     |   0.068   | 0.275  | __0.105__ |
|   Bm25l   |   Si    |    Si     |   0.069   | 0.279  | __0.106__ |
|   Bm25+   |   No    |    No     |   0.064   | 0.261  |  __0.1__  |
|   Bm25+   |   No    |    Si     |   0.066   | 0.265  | __0.102__ |
|   Bm25+   |   Si    |    No     |   0.067   | 0.273  | __0.104__ |
|   Bm25+   |   Si    |    Si     |   0.069   | 0.278  | __0.107__ |
|  Lucene   |   No    |    No     |   0.064   | 0.261  |  __0.1__  |
|  Lucene   |   No    |    Si     |   0.066   | 0.265  | __0.102__ |
|  Lucene   |   Si    |    No     |   0.067   | 0.273  | __0.104__ |
|  Lucene   |   Si    |    Si     |   0.069   | 0.278  | __0.107__ |

#### Micro promedios

|  Modelo   | Steamer | Stopwords | Precision | Recall |    F1     |
|:---------:|:-------:|:---------:|:---------:|:------:|:---------:|
| Robertson |   No    |    No     |   0.064   | 0.196  | __0.097__ |
| Robertson |   No    |    Si     |   0.066   |  0.2   | __0.099__ |
| Robertson |   Si    |    No     |   0.067   | 0.205  | __0.101__ |
| Robertson |   Si    |    Si     |   0.069   |  0.21  | __0.104__ |
|   Atire   |   No    |    No     |   0.064   | 0.196  | __0.097__ |
|   Atire   |   No    |    Si     |   0.066   |  0.2   | __0.099__ |
|   Atire   |   Si    |    No     |   0.067   | 0.205  | __0.101__ |
|   Atire   |   Si    |    Si     |   0.069   |  0.21  | __0.104__ |
|   Bm25l   |   No    |    No     |   0.064   | 0.197  | __0.097__ |
|   Bm25l   |   No    |    Si     |   0.066   |  0.2   | __0.099__ |
|   Bm25l   |   Si    |    No     |   0.068   | 0.206  | __0.102__ |
|   Bm25l   |   Si    |    Si     |   0.069   |  0.21  | __0.104__ |
|   Bm25+   |   No    |    No     |   0.064   | 0.196  | __0.097__ |
|   Bm25+   |   No    |    Si     |   0.066   |  0.2   | __0.099__ |
|   Bm25+   |   Si    |    No     |   0.067   | 0.205  | __0.101__ |
|   Bm25+   |   Si    |    Si     |   0.069   |  0.21  | __0.104__ |
|  Lucene   |   No    |    No     |   0.064   | 0.196  | __0.097__ |
|  Lucene   |   No    |    Si     |   0.066   |  0.2   | __0.099__ |
|  Lucene   |   Si    |    No     |   0.067   | 0.205  | __0.101__ |
|  Lucene   |   Si    |    Si     |   0.069   |  0.21  | __0.104__ |

Gracias a los datos obtenidos podemos concluir lo siguiente:

- Macro promedios -> se consigue un valor F1 de 0.107
    - Atire con steamer y stopwords
    - Lucene con steamer y stopwords
    - Bm25+ con steamer y stopwords

- Micro promedios -> se consigue un valor F1 de 0.104
    - Atire con steamer y stopwords
    - Bm25l con steamer y stopwords
    - Robertson con steamer y stopwords
    - Bm25+ con steamer y stopwords
    - Lucene con steamer y stopwords

Podemos concluir que tanto Atire, como Lucene y Bm25+, con configuración stemmer y stopwords, son buenos modelos, ambos tienen mismos valores de precision, recall y f1, y en términos de micro promedio, todos los modelos consiguen un f1 de 0.104

Debido a que hay varios modelos con el mismo f1, y en este caso de uso, elegiré el modelo Lucenen para el resto de ejercicios de la práctica

---

# Ejercicio 3

En este ejercicio se preparará un script para la realización bucle de expansión de consultas para el próximo ejercicio.

* Primero empezaremos ejecutando la consulta (al igual que se hace en ejercicios anteriores) y obteniendo las frecuencias de palabras de los documentos obtenidos


In [ ]:
from collections import Counter #Se requiere de esta libreria para el correcto funcionamiento
def frequency(corpus_tokenized):

    tmp = dict()

    for document in corpus_tokenized[0]:
        freqs = dict(Counter(document))
        for token, freq in freqs.items():
            try:
                tmp[token] += freq
            except:
                tmp[token] = freq

    inverted_vocab = {corpus_tokenized[1][key]: key for key in corpus_tokenized[1].keys()}

    total_freqs = dict()

    for key, freq in dict(tmp).items():
        term = inverted_vocab[key]
        total_freqs[term] = freq


    return total_freqs

* El siguiente paso es obtener las frecuencias de todos los documentos menos los obtenidos por la consulta, que lo haremos con la función anterior dando como parámetro el corpus tokenizado (para dicha tokenización se usara la función `tokenizado()`), ya que esta función de frecuencia nos la da el resultado para toda la colección es necesario hacer la resta de conjuntos

In [ ]:
def resta(dic1, dic2):
    result = dict()
    for key in dic1.keys():
        result[key] = dic1[key] - dic2.get(key, 0)
    return result

* Una vez obtenido esto, calculamos el LLR (Log Likelihood Ratio) para cada término

In [ ]:
import math

def x_log_x(x):
    return 0.0 if x == 0 else x * math.log(x)

def entropy(*elements):
    total = sum(elements)
    result = sum(x_log_x(element) for element in elements)
    return x_log_x(total) - result

def log_likelihood_ratio(k11, k12, k21, k22):
    row_entropy = entropy(k11 + k12, k21 + k22)
    column_entropy = entropy(k11 + k21, k12 + k22)
    matrix_entropy = entropy(k11, k12, k21, k22)
    if row_entropy + column_entropy < matrix_entropy:
        return 0.0
    return 2.0 * (row_entropy + column_entropy - matrix_entropy)

def root_log_likelihood_ratio(k11, k12, k21, k22):
    llr = log_likelihood_ratio(k11, k12, k21, k22)
    sqrt_llr = math.sqrt(llr)
    if (k11 / (k11 + k12)) < (k21 / (k21 + k22)):
        sqrt_llr = -sqrt_llr
    return sqrt_llr

* Por último, se ordenan los términos por su valor LLR y se seleccionan los m primeros términos, se declara como m, ya que será uno de los términos que se cambiara en el siguiente ejercicio



In [ ]:
def getBestWords(frequency, othersDocumentsFrequency, queryTokenize, m):
    relevantsWords = sum(frequency.values())
    othersWords = sum(othersDocumentsFrequency.values())

    relevances = dict()

    for word in frequency:
        k11 = frequency[word]
        k12 = relevantsWords - k11
        k21 = othersDocumentsFrequency[word]
        k22 = othersWords - k21

        llr = root_log_likelihood_ratio(k11, k12, k21, k22)
        relevances[word] = llr

    relevances = dict(sorted(relevances.items(), key=lambda item: item[1], reverse=True))

    result = []
    contador = 0

    for key in relevances.keys():
        if key not in queryTokenize:
            result.append(key)
            contador += 1
        if contador >= m:
            break

    return result

* Ahora solo nos quedaría añadir dichas palabras a la query principal y volver a ejecutar la consulta

In [ ]:
def updateQuery(query, bestWords):
    for word in bestWords:
        query += " "+word

    return query

Con estas funciones tendríamos el script necesario para realizar la expansión de consultas, para facilitar las llamadas a dichas funciones decidí englobarlas en una sola función que reciba los parámetros necesarios,

In [ ]:
def ejecucion(n,m):

    stemmer = Stemmer.Stemmer("english")
    stopwords = "en"

    corpus_plain,corpus_verbatim = prepareCorpus()
    corpus_tokenized = tokenizado(corpus_plain, stemmer, stopwords)

    retriever = createRetriever(corpus_verbatim, corpus_tokenized, method="lucene", idf_method="lucene")
    #retriever = openRetriever()

    query = "how contaminated are our children ?"  # primera query
    documents = processQueries(query, stemmer, stopwords, retriever, n)
    documentsTokenized = tokenizado([doc["text"] for doc in documents.documents[0]], stemmer, stopwords)

    fre = frequency(documentsTokenized)

    othersDocumentsFrequency = frequency(corpus_tokenized)

    othersDocumentsFrequency = resta(othersDocumentsFrequency, fre)

    bestWords = getBestWords(fre, othersDocumentsFrequency, tokenizado(query, stemmer, stopwords)[1], m)

    updateQuerry = updateQuery(query, bestWords)

    result = processQueries(updateQuerry, stemmer, stopwords, retriever, 100)

---
# Bibliografía
Enlaces consultado para la realización de los ejercicios:

## Ejercicio 1
- [Introduction to Lexical Search](https://colab.research.google.com/drive/1R4o4Mdt5mnByzFMjM9VihfOfbj3sNiVX?usp=sharing#scrollTo=OGGXn4zB1VIY)

## Ejercicio 2

- [Introduction to Lexical Search](https://colab.research.google.com/drive/1R4o4Mdt5mnByzFMjM9VihfOfbj3sNiVX?usp=sharing#scrollTo=OGGXn4zB1VIY)
- [Lexical Search and Performance Evaluation](https://colab.research.google.com/drive/1V3QvmZdKO6I3NOF_8_3r4oa9DQklsdWK?usp=sharing)